In [2]:
import cv2
from time import sleep

def middle(f1, f2):
    return f1 // 2 + f2 // 2

cap = cv2.VideoCapture('/Users/alexanderusatov/Documents/diploma/dog.mp4')
out = cv2.VideoWriter('/Users/alexanderusatov/Documents/diploma/avg.mp4', cv2.VideoWriter_fourcc(*'MJPG'), 25.0, (288, 360))

prev = None

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break
    if prev is None:
        prev = frame
        out.write(frame)
        continue
    out.write(middle(prev, frame))
    out.write(frame)
    prev = frame
cap.release()
out.release()
cv2.destroyAllWindows()
exit(0)

In [ ]:
from google.colab import drive
drive.mount("/content/gdrive")

import numpy as np
import cv2
from sys import exit

def perc_hash(f, y, x, r):
    h = np.zeros((2 * r + 1, 2 * r + 1, 3), dtype='uint8')
    for i in range(-r, r + 1):
        if y + r < 0 or y + r >= f.shape[0] or i == 0:
            continue
        for j in range(-r, r + 1):
            if x + r < 0 or x + r >= f.shape[1] or j == 0:
                continue
            h[r + i][r + j] = f[y + i][x + j]
    return h


# calculate delta between two hashes
def delta(h1, h2):
    sum_: int = 0
    for i in range(0, h1.shape[0]):
        for j in range(0, h1.shape[1]):
            for k in range(0, h1.shape[2]):
                sum_ += abs(int(h1[i][j][k]) - int(h2[i][j][k]))
    return sum_


# for point y x find new point y1 x1
def get_match(h, f, y, x, r):
    res = np.zeros(h.shape, dtype='uint8')
    yres, xres = y, x
    for i in range(max(y - r, 0), min(y + r, f.shape[0])):
        for j in range(max(x - r, 0), min(x + r, f.shape[1])):
            hfyx = perc_hash(f, y + i, x + j, r)
            if delta(h, hfyx) < delta(h, res):
                res, yres, xres = hfyx, i, j
    return yres, xres


# will return maxtrix: y1 x1 -> y2 x2
def get_new_coord(f1, f2, r):
    res = np.zeros((f1.shape[0], f1.shape[1], 2), dtype='uint16')
    for i in range(0, f1.shape[0]):
        for j in range(0, f1.shape[1]):
            res[i][j] = get_match(perc_hash(f2, i, j, r), f2, i, j, r)
    return res


def next_frame(f, d):
    res = f.copy()
    for i in range(f.shape[0]):
        for j in range(f.shape[1]):
            res[i][j] = f[i][j] // 2 + f[d[i][j][0]][d[i][j][1]] // 2
    return res


def middle(prev, frame, r):
    return next_frame(prev, get_new_coord(prev, frame, r))


cap = cv2.VideoCapture('gdrive/My Drive/original.mp4')
out = cv2.VideoWriter('gdrive/My Drive/res.mp4', cv2.VideoWriter_fourcc(*'MJPG'), 25.0, (288, 360))

prev = None

img = 1
while cap.isOpened():
    ret, frame = cap.read()
    if cv2.waitKey(1) == ord('q') or not ret:
        break
    if prev is None:
        prev = frame
        out.write(frame)
        continue
    out.write(middle(prev, frame, 1))
    out.write(frame)
    print(img)
    img += 1
    prev = frame
    
out.release()
cap.release()
cv2.destroyAllWindows()
print('end')
exit(0)


/Users/alexanderusatov/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:20: RuntimeWarning: overflow encountered in ubyte_scalars


In [1]:
def show_video(path : str, interval):
    import cv2
    from time import sleep
    
    cap = cv2.VideoCapture(path)
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret or cv2.waitKey(1) == ord('q'):
            break
        cv2.imshow(path.split('/')[-1], frame)
        sleep(interval)
    cv2.destroyAllWindows()
    exit(0)

In [2]:
show_video('/Users/alexanderusatov/Documents/diploma/original.mp4', 1 / 25)

In [ ]:
show_video('/Users/alexanderusatov/Documents/diploma/avg.mp4', 1 / 25)

In [ ]:
show_video('/Users/alexanderusatov/Documents/diploma/phash.mp4', 1 / 25)

In [1]:
def show_results(interval):
    import cv2
    from time import sleep
    
    avg, original, phash = '/Users/alexanderusatov/Documents/diploma/avg.mp4', \
                           '/Users/alexanderusatov/Documents/diploma/original.mp4', \
                           '/Users/alexanderusatov/Documents/diploma/phash.mp4'
    
    avg = cv2.VideoCapture(avg)
    original = cv2.VideoCapture(original)
    phash = cv2.VideoCapture(phash)
    frame_number = 0
    while True:
        ret1, frame1 = avg.read()
        if frame_number % 2 == 0:
            ret2, frame2 = original.read()
        ret3, frame3 = phash.read()
        frame_number += 1
        if not ret1 or not ret2 or not ret3 or cv2.waitKey(1) == ord('q'):
            break
        cv2.imshow('avg', frame1)
        cv2.imshow('original', frame2)
        cv2.imshow('phash', frame3)
        sleep(interval)
    cv2.destroyAllWindows()
    exit(0)

show_results(1 / 25)